In [26]:
import matplotlib.cm
import numpy as np
import pandas as pd
from statsmodels.stats.anova import AnovaRM
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from scipy.stats import ttest_rel
from scipy.stats import sem
import pingouin as pg
from constants import (ACCURACIES_PATH, ACC_UNSHIFTED_PATH, COND_ACC_PATH, RTs_PATH)
from utils.utils_statistical_analysis import stats_preprocessing

ValueError: A colormap named "polar" is already registered.

# Importing data and formatting it
Imports all the accuracies and brings them into a table that can be used for statistical analysis. Also segments the data to analyse only certain blocks.

In [2]:
accuracies_data = pd.read_csv(RTs_PATH, header=None)
stats_data = stats_preprocessing(accuracies_data)
cond_acc = pd.read_csv(COND_ACC_PATH, header=None)

#stats_data_pre = stats_data.loc[(stats_data['Run'] == 1) | (stats_data['Run'] == 2)]
#stats_data_treatment = stats_data.loc[(stats_data['Run'] == 3) | (stats_data['Run'] == 4)]
#stats_data_post = stats_data.loc[(stats_data['Run'] == 5) | (stats_data['Run'] == 6)]

#stats_data.info()

# Output
Exporting data to use in other programs

In [3]:
# np.savetxt("data/stats_data.csv", stats_data, delimiter=",")
# np.savetxt("data/stats_data_treatment.csv", stats_data_treatment, delimiter=",")
# np.savetxt("data/stats_data_pre.csv", stats_data_pre, delimiter=",")
# np.savetxt("data/stats_data_post.csv", stats_data_post, delimiter=",")

# ANOVA
calculates within-subjects ANOVA

In [4]:
model_1 = AnovaRM(data=stats_data, depvar='Accuracy', subject='Subject', within=['Run', 'Treatment']).fit()
print("\033[4m" + "Model 1" + "\033[0m")
print(model_1)
print("")

model_2 = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=stats_data)
print("\033[4m" + "Model 2" + "\033[0m")
print(model_2)
print("")

Model 1
                   Anova
              F Value Num DF  Den DF  Pr > F
--------------------------------------------
Run            0.3323 5.0000 145.0000 0.8928
Treatment      3.0567 1.0000  29.0000 0.0910
Run:Treatment  0.4922 5.0000 145.0000 0.7817


Model 2
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.006981      5    145  0.001396  0.332272  0.892792   
1        Treatment  0.067011      1     29  0.067011  3.056728  0.090984   
2  Run * Treatment  0.010457      5    145  0.002091  0.492234  0.781662   

   p-GG-corr       ng2       eps  
0   0.822885  0.000724  0.668456  
1   0.090984  0.006905  1.000000  
2   0.717608  0.001084  0.702364  



# T-Test
Calculates a t-test comparing individual runs to each other

In [5]:
sham_4 = stats_data.query('Treatment == 1 and Run == 4')['Accuracy']
stim_4 = stats_data.query('Treatment == 2 and Run == 4')['Accuracy']
print(np.mean(sham_4))
print(np.mean(stim_4))
print("")

sham_4 = sham_4 * 100
stim_4 = stim_4 * 100

t_test_1 = pg.ttest(sham_4, stim_4, paired=True, alternative='less')
print("\033[4m" + "T-Test 1" + "\033[0m")
print(t_test_1)

t_test_2 = ttest_rel(sham_4, stim_4, alternative='less')
print("\033[4m" + "T-Test 2" + "\033[0m")
print(t_test_2)

0.45336021547952904
0.429121383962198

T-Test 1
               T  dof alternative     p-val         CI95%   cohen-d   BF10  \
T-test  1.213677   29        less  0.882668  [-inf, 5.82]  0.135286  0.758   

           power  
T-test  0.008921  
T-Test 2
Ttest_relResult(statistic=1.2136771538287279, pvalue=0.8826682761520758)


# Performance Split
Splitting dataset into high and low performers and calculating ANOVA

In [6]:
subject_performance = stats_data.Accuracy.to_numpy()
subject_performance = np.mean(subject_performance.reshape(-1, 12), axis=1)
performance_index = np.argpartition(subject_performance, int(len(subject_performance)/2))
performance_index = performance_index + 1
performance_index = np.array_split(performance_index,2)

low_performers = stats_data[~(stats_data.Subject.isin(performance_index[1]))]
high_performers = stats_data[~(stats_data.Subject.isin(performance_index[0]))]

anova_low_performers = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=low_performers)
anova_high_performers = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=high_performers)

print("\033[4m" + "Low Performers" + "\033[0m")
print(np.mean(low_performers.Accuracy))
print(anova_low_performers)
print("")
print("\033[4m" + "High Performers" + "\033[0m")
print(np.mean(high_performers.Accuracy))
print(anova_high_performers)

Low Performers
0.3253641178775664
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.009985      5     70  0.001997  0.576575  0.717721   
1        Treatment  0.106428      1     14  0.106428  6.441448  0.023661   
2  Run * Treatment  0.002914      5     70  0.000583  0.263244  0.931640   

   p-GG-corr       ng2       eps  
0   0.610947  0.006789  0.521740  
1   0.023661  0.067908  1.000000  
2   0.846573  0.001991  0.584348  

High Performers
0.5583362822076106
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.033895      5     70  0.006779  1.438136  0.221450   
1        Treatment  0.001589      1     14  0.001589  0.061197  0.808205   
2  Run * Treatment  0.025260      5     70  0.005052  0.797532  0.555152   

   p-GG-corr       ng2       eps  
0   0.240364  0.010492  0.684611  
1   0.808205  0.000497  1.000000  
2   0.508445  0.007840  0.637507  


# Variance Split
Splitting dataset into subjects with high and low variance and calculating ANOVA

In [7]:
variances = accuracies_data.var(axis=1)
variance_index = np.argpartition(variances, int(len(variances)/2))
variance_index = variance_index + 1
variance_index = np.array_split(variance_index,2)

low_variance = stats_data[~(stats_data.Subject.isin(variance_index[0]))]
high_variance = stats_data[~(stats_data.Subject.isin(variance_index[1]))]

anova_low_variance = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=low_variance)
anova_high_variance = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=high_variance)

print("\033[4m" + "Low Variance" + "\033[0m")
print("Mean Accuracy = " + str(np.mean(low_variance.Accuracy)))
print("Mean Variance = " + str(np.mean(variances[variance_index[0] - 1])))
print(anova_low_variance)
print("")
print("\033[4m" + "High Variance" + "\033[0m")
print("Mean Accuracy = " + str(np.mean(high_variance.Accuracy)))
print("Mean Variance = " + str(np.mean(variances[variance_index[1] - 1])))
print(anova_high_variance)

Low Variance
Mean Accuracy = 0.48908157525182344
Mean Variance = 0.00237771773748847
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.026567      5     70  0.005313  0.849993  0.519146   
1        Treatment  0.139385      1     14  0.139385  4.133080  0.061467   
2  Run * Treatment  0.047446      5     70  0.009489  1.544181  0.187426   

   p-GG-corr       ng2       eps  
0   0.481653  0.004059  0.651153  
1   0.061467  0.020936  1.000000  
2   0.213362  0.007226  0.650288  

High Variance
Mean Accuracy = 0.3946188248333534
Mean Variance = 0.009413812458984146
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.014054      5     70  0.002811  1.424925  0.226054   
1        Treatment  0.000053      1     14  0.000053  0.008076  0.929668   
2  Run * Treatment  0.020872      5     70  0.004174  2.281512  0.055709   

   p-GG-corr       ng2       eps  
0   0.251480  0.006487  0.550307  
1   0.92

# Comparing day 1 to day 2
Using the unshifted data to test if there is a difference in the performance between day 1 and day 2 and to see if there is a learning effect

In [8]:
accuracies_unshifted_data = pd.read_csv(ACC_UNSHIFTED_PATH, header=None)
stats_data_unshifted = stats_preprocessing(accuracies_unshifted_data)

stats_data_day1 = stats_data.loc[(stats_data['Treatment'] == 1)]
stats_data_day2 = stats_data.loc[(stats_data['Treatment'] == 2)]

unshifted_anova = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=stats_data_unshifted)
anova_day1 = pg.rm_anova(dv='Accuracy', within=['Run'], subject='Subject', data=stats_data_day1)
anova_day2 = pg.rm_anova(dv='Accuracy', within=['Run'], subject='Subject', data=stats_data_day2)

print("\033[4m" + "ANOVA comparing day 1 and 2" + "\033[0m")
print(unshifted_anova)
print("")
print("\033[4m" + "ANOVA of day 1" + "\033[0m")
print(anova_day1)
print("")
print("\033[4m" + "ANOVA of day 2" + "\033[0m")
print(anova_day2)

ANOVA comparing day 1 and 2
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.025461      5    155  0.005092  0.713259  0.614357   
1        Treatment  0.000086      1     31  0.000086  0.009457  0.923155   
2  Run * Treatment  0.037047      5    155  0.007409  0.750078  0.587216   

   p-GG-corr       ng2       eps  
0   0.599474  0.005739  0.895642  
1   0.923155  0.000020  1.000000  
2   0.560758  0.008330  0.806473  

ANOVA of day 1
  Source  ddof1  ddof2         F     p-unc       ng2       eps
0    Run      5    145  0.330488  0.893899  0.001391  0.773856

ANOVA of day 2
  Source  ddof1  ddof2        F     p-unc  p-GG-corr       ng2       eps  \
0    Run      5    145  0.47381  0.795327   0.718755  0.002137  0.658523   

   sphericity   W-spher   p-spher  
0       False  0.227741  0.000258  


# Comparing Baseline to Stimulation Phase
Calculating the ANOVA comparing baseline trials with trial 3 and 4. First on both days, then only on the day of the actual stimulation.

In [9]:
#stats_data_short = stats_data.loc[(stats_data['Run'] == 1) | (stats_data['Run'] == 2) | (stats_data['Run'] == 3) | (stats_data['Run'] == 4)]
stats_data.Run = stats_data.Run.replace(2, 1)
anova_short = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=stats_data)

stats_data_treatment = stats_data.loc[(stats_data['Treatment'] == 2)]
anova_short_treatment = pg.rm_anova(dv='Accuracy', within=['Run'], subject='Subject', data=stats_data_treatment)

#tukey = pairwise_tukeyhsd(endog=stats_data_treatment['Accuracy'],
#                          groups=stats_data_treatment['Run'],
#                          alpha=0.05)

print("\033[4m" + "ANOVA comparing Baseline to run 3 and 4" + "\033[0m")
print(anova_short)
print("")
print("\033[4m" + "ANOVA comparing Baseline to run 3 and 4 in treatment condition" + "\033[0m")
print(anova_short_treatment)
print("")
#print("\033[4m" + "Pairwise comparisons" + "\033[0m")
#print(tukey)

ANOVA comparing Baseline to run 3 and 4
            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.006485      4    116  0.001621  0.453392  0.769744   
1        Treatment  0.047746      1     29  0.047746  2.446196  0.128658   
2  Run * Treatment  0.004268      4    116  0.001067  0.317811  0.865574   

   p-GG-corr       ng2       eps  
0   0.724594  0.000804  0.786447  
1   0.128658  0.005892  1.000000  
2   0.819998  0.000530  0.779716  

ANOVA comparing Baseline to run 3 and 4 in treatment condition
  Source  ddof1  ddof2         F     p-unc  p-GG-corr       ng2       eps  \
0    Run      4    116  0.471339  0.756674   0.696558  0.001705  0.724617   

   sphericity   W-spher  p-spher  
0       False  0.398652  0.00279  



# Comparing Sham to Stimulation Phase
Calculating ANOVA comparing run 3 and 4 of the sham day with run 3 and 4 of the stimulation day

In [10]:
stats_data_sham_stim = stats_data.loc[(stats_data['Run'] == 3) | (stats_data['Run'] == 4)]
stats_data_sham_stim.loc[(stats_data['Treatment'] == 1)].Run = stats_data_sham_stim.loc[(stats_data['Treatment'] == 1)].Run.replace(4, 3)

anova_sham_stim = pg.rm_anova(dv='Accuracy', within=['Run', 'Treatment'], subject='Subject', data=stats_data_sham_stim)
print(anova_sham_stim)

            Source        SS  ddof1  ddof2        MS         F     p-unc  \
0              Run  0.000309      1     29  0.000309  0.078438  0.781413   
1        Treatment  0.020611      1     29  0.020611  2.223811  0.146696   
2  Run * Treatment  0.000117      1     29  0.000117  0.026812  0.871068   

   p-GG-corr       ng2  eps  
0   0.781413  0.000093  1.0  
1   0.146696  0.006145  1.0  
2   0.871068  0.000035  1.0  


/var/folders/h8/2kfl3g3j6qb3s84n6r41gc8m0000gn/T/ipykernel_66773/2712407801.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stats_data_sham_stim.loc[(stats_data['Treatment'] == 1)].Run = stats_data_sham_stim.loc[(stats_data['Treatment'] == 1)].Run.replace(4, 3)


In [11]:
print(stats_data_sham_stim.loc[(stats_data['Treatment'] == 1)].Run)

2      3
3      4
14     3
15     4
26     3
27     4
38     3
39     4
50     3
51     4
62     3
63     4
74     3
75     4
86     3
87     4
98     3
99     4
110    3
111    4
122    3
123    4
134    3
135    4
146    3
147    4
158    3
159    4
170    3
171    4
182    3
183    4
194    3
195    4
206    3
207    4
218    3
219    4
230    3
231    4
242    3
243    4
254    3
255    4
266    3
267    4
278    3
279    4
290    3
291    4
302    3
303    4
314    3
315    4
326    3
327    4
338    3
339    4
350    3
351    4
Name: Run, dtype: category
Categories (5, int64): [1, 3, 4, 5, 6]


# Descriptives on individual subjects
General statistics of individual subjects needed for the results section

In [12]:
# get mean and SD of single participant
subject = 26
print(accuracies_data.iloc[subject].mean())
print(accuracies_data.iloc[subject].std())

# get mean and SD of all participants
print(accuracies_data.stack().mean())
print(accuracies_data.stack().std())
print(sem(accuracies_data.stack()))

# get mean and SD of day 1 or day 2
# print(stats_data_day1["Accuracy"].mean())
# print(stats_data_day2["Accuracy"].mean())
# print(stats_data_day1["Accuracy"].std())
# print(stats_data_day2["Accuracy"].std())

# age of the participants
# age = [28, 24, 23, 23, 23, 28, 23, 26, 25, 24, 27, 27, 23, 26, 21, 25, 24, 36, 24, 26, 26, 26, 29, 34, 24, 22, 34, 25, 28, 26, 25, 26]
# print(len(age))
# print(np.mean(age))
# print(np.median(age))
# print(np.std(age))
# print(min(age))
# print(max(age))

0.17537608806944446
0.05333091492477195
0.4418502000425884
0.16456422022496447
0.008673295954673909


Comparing accuracy for remember stimulus 1 and remember stimulus 2

In [13]:
print(np.mean(cond_acc.loc[0]))
print(np.mean(cond_acc.loc[1]))
print(np.std(cond_acc.loc[0]))
print(np.std(cond_acc.loc[1]))

cond_ttest = pg.ttest(cond_acc.loc[0], cond_acc.loc[1], paired=False)
print("\033[4m" + "Conditional T-Test" + "\033[0m")
print(t_test_1)

0.6335468219668099
0.6330849320361348
0.06394476211463561
0.06034410459973374
Conditional T-Test
               T  dof alternative     p-val         CI95%   cohen-d   BF10  \
T-test  1.213677   29        less  0.882668  [-inf, 5.82]  0.135286  0.758   

           power  
T-test  0.008921  


# Statistically test running mean

In [42]:
import mne
from mne import io
from mne.stats import permutation_cluster_test
print("at least")

condition_sham = stats_data.Accuracy.loc[stats_data['Treatment'] == 1]
condition_treat = stats_data.Accuracy.loc[stats_data['Treatment'] == 2]

T_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test([condition_treat, condition_sham], tail=0, out_type='mask')

at least


/var/folders/h8/2kfl3g3j6qb3s84n6r41gc8m0000gn/T/ipykernel_66773/442537960.py:9: RuntimeWarning: Ignoring argument "tail", performing 1-tailed F-test
  T_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test([condition_treat, condition_sham], tail=0, out_type='mask')
/Users/lucakammer/.conda/envs/data_analysis/lib/python3.9/site-packages/mne/stats/cluster_level.py:852: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  X = [x[:, np.newaxis] if x.ndim == 1 else x for x in X]
/var/folders/h8/2kfl3g3j6qb3s84n6r41gc8m0000gn/T/ipykernel_66773/442537960.py:9: RuntimeWarning: No clusters found, returning empty H0, clusters, and cluster_pv
  T_obs, clusters, cluster_p_values, H0 = mne.stats.permutation_cluster_test([condition_treat, condition_sham], tail=0, out_type='mask')
